In [11]:
import qutip as qt
import numpy as np
from qutip import *
from scipy import *
import imageio
from matplotlib import *
import matplotlib as mpl

from qutip.operators import sigmax, sigmay, sigmaz

def extract_vec(v):
    v=[v]
    vector=[]
    for i in v:
        vector.append([
            expect(sigmax(),i),
            expect(sigmay(),i),
            expect(sigmaz(),i)     
        ])
    return(vector)

σx = sigmax()
σy = sigmay()
σz = sigmaz()

## Loading the Julia simulation data

In [14]:
def animate_bloch_vectors(loc, sxlists, sylists, szlists, duration=0.1, save_all=False):
    b = Bloch()
    b.figsize = [10,10]
    b.view = [-70,30]

    # Assuming all Bloch vectors have the same length
    length = len(sxlists[0])

    ## normalize colors to the length of data ##
    nrm = mpl.colors.Normalize(0, length)
    from matplotlib import cm
    colors = cm.cool(nrm(range(length)))
    alpha_value = 0.1  # Set this to your desired transparency level between 0 and 1
    colors1 = [(r, g, b, alpha_value) for r, g, b, _ in colors]

    ## customize sphere properties ##
    b.point_color = list(colors1)
    b.point_marker = ['o']
    b.point_size = [10] * len(sxlists)

    images = []
    for i in range(length):
        b.clear()
        
        avg_sx = np.sum([sxlists[j][i] for j in range(len(sxlists))]) / len(sxlists)
        avg_sy = np.sum([sylists[j][i] for j in range(len(sylists))]) / len(sylists)
        avg_sz = np.sum([szlists[j][i] for j in range(len(szlists))]) / len(szlists)
        
        # Add vectors with the desired colors
        colors_for_all_vectors = ['r'] * len(sxlists) + ['k']  # 'b' is for the average vector
        b.vector_color = colors_for_all_vectors
        for j in range(len(sxlists)):
            b.add_vectors([sxlists[j][i], sylists[j][i], szlists[j][i]])
            # b.add_points([sxlists[j][:(i+1)], sylists[j][:(i+1)], szlists[j][:(i+1)]], meth='l')
        
        # Add the average vector
        b.add_vectors([avg_sx, avg_sy, avg_sz])
        
        if save_all:
            b.save(dirc='tmp')
            filename = "tmp/bloch_multi_%01d.png" % i
        else:
            filename = 'temp_file_multi.png'
            b.save(filename)
        images.append(imageio.imread(filename))
    fps = 1.0 / duration
    imageio.mimsave(loc, images, fps=fps)

    # imageio.mimsave(loc, images)


In [31]:
import h5py
import numpy as np

with h5py.File('./data/rho_init_plus_i_DD_time_dependent_XX_under_fixed_phases.h5', 'r') as file:
    # Read the 'free' group datasets
    free_group = file['free']
    rho_all_traj_list = [np.array(free_group[f'dataset_{i}']) for i in range(1, len(free_group)+1)]
    
    # Read the 'DD' group datasets
    DD_group = file['XX']
    rho_all_traj_list_XX = [np.array(DD_group[f'dataset_{i}']) for i in range(1, len(DD_group)+1)]
    
    # Read the 'time' group datasets
    time_group = file['time']
    t_axis = [time_group[f'dataset_{i}'][()] for i in range(1, len(time_group)+1)]


In [36]:
data = rho_all_traj_list_XX
data_qobj = []
for i in range(len(data)):
    el_list = [] 
    for j in range(len(data[i])):
        el = Qobj(data[i][j], dims=[[2], [1]])
        el_list.append(el)
    data_qobj.append(el_list)


sx_list_lists = []
sy_list_lists = []
sz_list_lists = []
for i1 in range(len(data_qobj)):
    data_rho = data_qobj[i1]
    AA = [extract_vec(data_rho[i]) for i in range(len(data_rho))]
    sx_list = []
    sy_list = []
    sz_list = []
    for el in AA:
        sx_val,sy_val,sz_val = el[0]
        sx_list.append(sx_val)
        sy_list.append(sy_val)
        sz_list.append(sz_val)
    sx_list_lists.append(sx_list)
    sy_list_lists.append(sy_list)
    sz_list_lists.append(sz_list)

animate_bloch_vectors("./animations/rho_init_plus_i_DD_XX_under_fixed_phases.mp4",sx_list_lists,sy_list_lists,sz_list_lists, duration=0.1, save_all=False)